In [ ]:
!pip install pyitlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np
from pyitlib import discrete_random_variable as drv
import pandas as pd
import math

In [ ]:
train = np.load("/content/drive/My Drive/VGG16_train_features.npy")
print(train.shape)

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

In [ ]:
b = []
for i in range(1152):
  a=[]
  if(i<8):
    for j in range(i):
      a.append(j)
    for j in range(i+1,8):
      a.append(j)
    print(a)
    b.append(a)
  elif (i>=1148):
    for j in range(1144,i):
      a.append(j)
    for j in range(i+1,1152):
      a.append(j)
    print(a)
    b.append(a)
  else:
    for j in range(i-4,i):
      a.append(j)
    for j in range(i+1,i+4):
      a.append(j)
    print(a)
    b.append(a)
print(b)

In [ ]:
import pandas as pd
dataframe_train = pd.read_csv("/content/drive/My Drive/train_feature_csv")
y_train = dataframe_train['label']
y_train = y_train.to_numpy()
print(y_train)

In [ ]:
for i in range(1000,1152):
  X_i = train[:,i]
  shap_score = 0
  classifier_list =list(powerset(b[i]))
  for k in range(len(classifier_list)):
    classifier_subset = list(classifier_list[k])
    m_info_classif = 0
    m_info_class = drv.information_mutual(X_i, y_train)
    for j in range(len(classifier_subset)):
      X_j= train[:,classifier_subset[j]]
      temp = drv.information_mutual(X_i, X_j)
      m_info_classif = m_info_classif + temp
    m_info_classif = m_info_classif/len(classifier_subset)
    marginal_info = m_info_class-m_info_classif
    shap_score = shap_score + math.factorial(len(classifier_subset))*math.factorial((8 - len(classifier_subset) -1))*marginal_info/math.factorial(8)
  print(shap_score)
  shapley_value.append(shap_score)

In [ ]:
minimum = min(shapley_value)
sh = shapley_value/minimum
sh = [ -x for x in sh]

In [ ]:
features = []
for i in range(len(sh)):
  features.append([i,sh[i]])

def Sort(sub_li): 
    sub_li.sort(key = lambda x: x[1],reverse=True) 
    return sub_li

f = Sort(features)
selected = f[0:750]

In [ ]:
numbers=[]
for i in range(len(selected)):
  numbers.append(selected[i][0])

In [ ]:
x_train_selected = []
for i in range(len(numbers)):
  x_train_selected.append(train[:,numbers[i]])

x_train_selected = np.transpose(x_train_selected)
x_train_selected.shape

In [ ]:
a = x_train_selected
a.shape

In [ ]:
!pip install cur

In [ ]:
r = 300
from cur import cur_decomposition
C, U, k = cur_decomposition(a, r)
print(a.shape)
print(k.shape)
print(C.shape)

In [ ]:
similarity = np.zeros([300,300])
for i in range(300):
  for j in range(300):
    similarity[i][j] = np.dot(k[i],k[j])/(np.linalg.norm(k[i])*np.linalg.norm(k[j]))
print(similarity)

In [ ]:
I=np.identity(750)
o=np.ones([750,750])/750
temp=I-o
temp1=np.dot(temp,similarity)
similarity=np.dot(temp1,temp)

In [ ]:
eig_val,eig_vec = np.linalg.eig(similarity)
eig_val = np.real(eig_val)
eig_vec = np.real(eig_vec)
sig = np.diag(eig_val)similarity1 = np.zeros([3975,300])
for i in range(3975):
  for j in range(300):
    similarity1[i][j] = np.dot(a[i],k[j])/(np.linalg.norm(a[i])*np.linalg.norm(k[j]))
print(similarity1)

In [ ]:
temp = np.dot(similarity1,eig_vec)
representation = np.dot(temp,np.linalg.inv(sig))
print(representation)

In [ ]:
np.save('/content/' +'REPRESENTATION_2500_K'+'.npy', representation)

In [ ]:
representation = a
representation_train = representation[0:3086]
representation_test = representation[3086:3206]
representation_val = representation[3206:3975]
print(representation_test.shape)
print(representation_train.shape)

In [ ]:
import pandas as pd
dataframe_train = pd.read_csv("/content/drive/My Drive/train_feature_csv")
y_train = dataframe_train['label']
y_train = y_train.to_numpy()
print(y_train)

In [ ]:
import pandas as pd
dataframe_test = pd.read_csv("/content/drive/My Drive/test_features_csv")
y_test = dataframe_test['label']
y_test = y_test.to_numpy()
print(y_test)
print(y_test.shape)

In [ ]:
import pandas as pd
dataframe_test = pd.read_csv("/content/drive/My Drive/val_feature_csv")
y_val = dataframe_test['label']
y_val = y_val.to_numpy()
print(y_val)
print(y_val.shape)

In [ ]:
import keras
y_train = keras.utils.to_categorical(y_train, num_classes=3)
y_val = keras.utils.to_categorical(y_val, num_classes=3)
y_test = keras.utils.to_categorical(y_test)
print(y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
# 32 .4 16 .5 3 (90)
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax',name= 'output'))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
representation_train = sc.fit_transform(representation_train)
representation_test = sc.fit_transform(representation_test)
representation_val = sc.fit_transform(representation_val)

In [ ]:
adam_opt=Adam(lr = 0.001, beta_1=0.6, beta_2=0.8,amsgrad=True)

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(representation_train,y_train, epochs=500, batch_size=32, validation_data=(representation_test,y_test))

In [ ]:
import pandas as pd
dataframe_test = pd.read_csv("/content/drive/My Drive/test_feature_csv")
y_test = dataframe_test['label']
y_test = y_test.to_numpy()
print(y_test)
print(y_test.shape)

In [ ]:
model.predict(representation_test)